Entrenamiento del modelo:

entrenaremos tres distintos modelos utilizando los algoritmos del Framework Scikit-learn: Gradient Boosting, Extra Trees y Random Forest

En lugar de crear tres scripts distintos para cada algoritmo, crearemos uno sólo el cual a partir de los parámetros recibidos, entrenará el modelo con el algoritmo indicado. Utilizaremos la misma técnica de k-fold Cross-Validation

In [ ]:
training_script_file = 'code/train_and_serve.py'


In [ ]:
 %%writefile $training_script_file
import argparse
import pickle
import os
import io
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

# Carga el modelo en memoria
def model_fn(model_dir):
    print('Cargando modelo: model_fn')
    clf = read_pkl(os.path.join(model_dir, "model.pkl"))
    return clf

# Deserealiza el body de la petición para poder generar las predicciones
def input_fn(request_body, request_content_type):

    if request_content_type == 'application/json':
        input_data = json.loads(request_body)
        input_data = pd.DataFrame.from_dict(input_data)

        return input_data
        
    elif request_content_type == 'text/csv':      
        input_data = io.StringIO(request_body)        
        return pd.read_csv(input_data, header=None)
    else:
        raise ValueError("El endpoint del modelo solamente soporta Content-Types: 'application/json' o 'text/csv' como entrada")
                
# Genera la predicción sobre el objeto deserializado, con el modelo previamente cargado en memoria
def predict_fn(input_data, model):
    predict_proba = getattr(model, 'predict_proba', None)
    if callable(predict_proba):
        return predict_proba(input_data)[:, 1]
    else:
        return model.predict(input_data)

# Serializa el resultado de la predicción al correspondiente content type deseado
def output_fn(predictions, response_content_type):
    if response_content_type == 'application/json':        
        return json.dumps(predictions.tolist())
    elif response_content_type == 'text/csv':
        predictions_response = io.StringIO()
        np.savetxt(predictions_response, predictions, delimiter=',')
        return predictions_response.getvalue()
    else:
        raise ValueError("El endpoint del modelo solamente soporta Content-Types: 'application/json' o 'text/csv' como respuesta")        
        
def read_pkl(file):
    with open(file, 'rb') as f:
        return pickle.load(f)
    
def to_pkl(data, file):
    with open(file, 'wb') as f:
        pickle.dump(data, f)

def random_forest(**hyperparameters):
    return RandomForestClassifier(n_jobs=-1, 
                                  min_samples_split=hyperparameters['min_samples_split'],
                                  n_estimators=hyperparameters['n_estimators'],
                                  max_depth=hyperparameters['max_depth'],
                                  max_features=hyperparameters['max_features'])

def gradient_boosting(**hyperparameters):
    return GradientBoostingClassifier(learning_rate=hyperparameters['learning_rate'],
                                      min_samples_split=hyperparameters['min_samples_split'],
                                      n_estimators=hyperparameters['n_estimators'],
                                      max_depth=hyperparameters['max_depth'],
                                      max_features=hyperparameters['max_features'])

def extra_trees(**hyperparameters):
    return ExtraTreesClassifier(n_jobs=-1, 
                                min_samples_split=hyperparameters['min_samples_split'],
                                n_estimators=hyperparameters['n_estimators'],
                                max_depth=hyperparameters['max_depth'],
                                max_features=hyperparameters['max_features'])

def invalid_algorithm(**hyperparameters):
    raise Exception('Invalid Algorithm')
    
def algorithm_selector(algorithm, **hyperparameters):
    algorithms = {
        'RandomForest': random_forest,
        'GradientBoosting': gradient_boosting,
        'ExtraTrees': extra_trees
    }
    
    clf = algorithms.get(algorithm, invalid_algorithm)    
    return clf(**hyperparameters)


if __name__=='__main__':
    script_name = os.path.basename(__file__)
    print(f'INFO: {script_name}: Iniciando entrenamiento del modelo')
    
    parser = argparse.ArgumentParser()
    
    parser.add_argument('--output-data-dir', type=str, default=os.environ.get('SM_OUTPUT_DATA_DIR'))
    parser.add_argument('--model-dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train-data', type=str, default=os.environ.get('SM_CHANNEL_TRAIN_DATA'))
    parser.add_argument('--train-target', type=str, default=os.environ.get('SM_CHANNEL_TRAIN_TARGET'))
    
    parser.add_argument('--algorithm', type=str)
    parser.add_argument('--splits', type=int, default=10)
    parser.add_argument('--target-metric', type=str)
    
    parser.add_argument('--learning-rate', type=float)
    parser.add_argument('--min-samples-split', type=int)
    parser.add_argument('--n-estimators', type=int)
    parser.add_argument('--max-depth', type=int)
    parser.add_argument('--max-features', type=int)
    
            
    args, _ = parser.parse_known_args()
    
    print(f'INFO: {script_name}: Parametros recibidos: {args}')

    # Cargar datasets
    files = os.listdir(args.train_data)
    if len(files) == 1:
        train_data = pd.read_csv(os.path.join(args.train_data, files[0]))
    else:
        raise Exception()
    
    files = os.listdir(args.train_target)
    if len(files) == 1:
        train_target = pd.read_csv(os.path.join(args.train_target, files[0]))
        train_target = train_target['dataset_credit_risk'].tolist()
    else:
        raise Exception()

     
    clf = algorithm_selector(args.algorithm, 
                             learning_rate=args.learning_rate,
                             min_samples_split=args.min_samples_split,
                             n_estimators=args.n_estimators,
                             max_depth=args.max_depth,
                             max_features=args.max_features)
    
    skf = StratifiedKFold(n_splits=args.splits)    
    cv_scores = cross_validate(clf, train_data, train_target, cv=skf, scoring=args.target_metric, n_jobs=-1)
    print('{} = {}%'.format(args.target_metric, cv_scores['test_score'].mean().round(4)*100))
    
    # Entrenar el modelo
    clf.fit(train_data, train_target) 
    
    # Guardar modelo
    to_pkl(clf, os.path.join(args.model_dir, 'model.pkl'))

    print(f'INFO: {script_name}: Finalizando el entrenamiento del modelo')   



Empaquetamos el script en un archvio .tar.gz y lo subimos a un bucket de Amazon S3 para poder utilizarlo en el job de entrenamiento y posteriormente para el despliegue del modelo.

In [ ]:
training_script_tar_file = os.path.join('code',os.path.splitext(os.path.basename(training_script_file))[0] + '.tar.gz')

sagemaker_utils.create_tar_gz(training_script_file, training_script_tar_file)

training_script_path = sagemaker_utils.upload(training_script_tar_file, f's3://{bucket}/{code_prefix}')


La función model_fn tiene la lógica para cargar el modelo en memoria y se ejecuta solamente al iniciar por primera vez el contenedor, una vez hecho el despligue del modelo.
Con cada petición al endpoint del modelo, una vez desplegado, se ejecutan las siguientes funciones en este respectivo orden:
input_fn la cual recibe los datos provenientes de la petición y se encarga de realizar la transformación que sea necesaria para poder posteriormente pasar los datos al modelo para generar la predicción
predict_fn se encarga de generar la predicción, recibiendo como entrada la salida de la función input_fn
output_fn una vez obtenida la predicción, esta función se encarga de realizar las transformaciones necesarias para devolver la respuesta al solicitante de la petición

Crear Training Job


Para crear el Job de entrenamiento en Amazon SageMaker utilizamos la clase CustomEstimator para crear un Estimator el cual permita integrar nuestro script train_and_serve.py con el contenedor Docker que previamente creamos para el entrenamiento de nuestros modelos.

También utilizaremos la expresión regular recall = (\d+\.\d{1,2})? para definir una métrica de desempeño de nuestro algoritmo, en este caso por tratarse de un problema de riesgo, lo que buscamos es incrementar el Recall, esta métrica es calculada mediante el uso de k-Fold Cross-Validation. Amazon SageMaker aplica esta expresión regular a los mensajes de la salida estándar. Posteriormente mediante el uso de esta métrica podremos crear un Job de optimización de hiperparámetros.


Para ejecutar los procesos de entrenamiento lo haremos llamando el método fit del estimator previamente creado. Y lo haremos creando tres procesos de entrenamiento en paralelo


In [ ]:
estimators = {'GradientBoosting':{}, 'RandomForest':{}, 'ExtraTrees':{}}
metric_name = 'cross-val:recall'
metric_regex = 'recall = (\d+\.\d{1,2})?'

for algorithm in estimators:   
    estimators[algorithm] = Estimator(
        image_uri = docker_images['Training']['image_uri'],        
        entry_point = os.path.basename(training_script_file),
        source_dir = training_script_path,
        role = sagemaker_role,
        instance_count = 1,
        instance_type = 'ml.m5.xlarge',
        output_path = f's3://{bucket}/{model_prefix}',
        metric_definitions = [{'Name': metric_name, 'Regex': metric_regex}],
        volume_size = 5,
        max_run = 60*60*2, # dos horas
        hyperparameters={
            'algorithm':algorithm,
            'splits':5,
            'target-metric':'recall',
            'learning-rate': 0.1, 
            'min-samples-split': 3, 
            'n-estimators': 300,
            'max-depth': 25,
            'max-features':20})
    
    estimators[algorithm].fit(
        {'train_data': sagemaker_utils.get_processor_output_path(processor, 'train_data'),
        'train_target': sagemaker_utils.get_processor_output_path(processor, 'train_target')},
        wait=False)


In [ ]:
#definimos un waiter
sagemaker_utils.wait_for_training_jobs(estimators)


para obtener las métricas de desempeño de cada estimator, podemos iterar a través de estos.

In [ ]:
for estimator in estimators:
    metrics = estimators[estimator].training_job_analytics.dataframe()
    test_recall = metrics[metrics['metric_name'] == metric_name]['value'].values[0]
    print(f'{estimator}: cross-val:recall = {test_recall}%')
